In [1]:
import os

import numpy as np
import pandas as pd
import zipfile
import tensorflow as tf
import shutil
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = os.path.abspath("./drive/MyDrive/MegaClassifier")

Mounted at /content/drive


In [3]:
MODEL_NAME = "MegaClassifier_c"
VERSION = "v1"

In [4]:
DATASETS = {
    "MegaClassifier_a": "onlyDetectionsForTrain",
    "MegaClassifier_b": "emptyOriginalAnimalDetection",
    "MegaClassifier_c": "emptyNonEmptyDataset",
}

In [5]:
DATASET_PATH_ZIP = os.path.join(BASE_PATH, f"data/processed/{DATASETS[MODEL_NAME]}.zip")
with zipfile.ZipFile(DATASET_PATH_ZIP, 'r') as zip_ref:
      zip_ref.extractall("./data/processed")
DATASET_DIR = os.path.abspath(f"data/processed/{DATASETS[MODEL_NAME]}")

In [6]:
EPOCHS = 10

IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [ ]:
for BATCH_INDEX in range(4):

  OPTIMIZER = tf.keras.optimizers.Adam()

  LOSS_FUNCTION = tf.keras.losses.BinaryCrossentropy()

  METRICS = [
      BinaryAccuracy(name="accuracy"),
      Precision(name="precision"),
      Recall(name="recall"),
      AUC(name="auc"),
  ]

  BATCH_SIZES = [16, 32, 64, 128]
  BATCH_SIZE = BATCH_SIZES[BATCH_INDEX]

  SUBVERSION = VERSION + f".{BATCH_INDEX}"

  LOGS_PATH = os.path.abspath(f"./logs")
  if os.path.exists(LOGS_PATH):
    !rm -rf {LOGS_PATH}
  os.makedirs(LOGS_PATH, exist_ok=True)

  MODELS_PATH = os.path.abspath("./models")
  if os.path.exists(MODELS_PATH):
    !rm -rf {MODELS_PATH}
  os.makedirs(MODELS_PATH, exist_ok=True)


  train_datagen = ImageDataGenerator(
      preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
  )
  train_images = train_datagen.flow_from_directory(
    directory=f"{DATASET_DIR}/train",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=['vacia', 'animal'],
    shuffle=True,
    seed=SEED,
  )

  datagen = ImageDataGenerator(
      preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
  )
  validation_images = datagen.flow_from_directory(
    directory=f"{DATASET_DIR}/validation",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    classes=['vacia', 'animal'],
    shuffle=False,
    seed=SEED,
  )

  pretrained_model = tf.keras.applications.EfficientNetB5(
      weights="imagenet",
      include_top=False,
      input_shape=IMAGE_SHAPE,
  )
  pretrained_model.trainable = False

  model = tf.keras.Sequential(
    [
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ],
    name=f"{MODEL_NAME}_{SUBVERSION}",
  )

  model.compile(
      optimizer=OPTIMIZER,
      loss=LOSS_FUNCTION,
      metrics=METRICS,
  )

  history = model.fit(
      train_images,
      epochs=EPOCHS,
      batch_size=BATCH_SIZE,
      validation_data=validation_images,
  )

  dataframe = pd.DataFrame(history.history)
  history_path = os.path.join(LOGS_PATH, f"history_{SUBVERSION}.csv")
  dataframe.to_csv(history_path, sep=";", index=False)


  model.save(os.path.join(MODELS_PATH,f"{MODEL_NAME}_{SUBVERSION}.weights.h5"))
  model.save(os.path.join(MODELS_PATH,f"{MODEL_NAME}_{SUBVERSION}.keras"))

  shutil.copytree(LOGS_PATH, os.path.join(BASE_PATH,f"logs/{MODEL_NAME}/{VERSION}/{SUBVERSION}"),dirs_exist_ok=True)
  shutil.copytree(MODELS_PATH, os.path.join(BASE_PATH,f"models/{MODEL_NAME}/{VERSION}/{SUBVERSION}"),dirs_exist_ok=True)

Found 19995 images belonging to 2 classes.
Found 4286 images belonging to 2 classes.
115263384/115263384 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
  50/1250 ━━━━━━━━━━━━━━━━━━━━ 2:28 124ms/step - accuracy: 0.7785 - auc: 0.8351 - loss: 0.5164 - precision: 0.8102 - recall: 0.8431